In [45]:
# import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import os
import numpy as np

In [2]:
# JSON files from Kaggle were saved to github for retrevial
# https://www.kaggle.com/c/whats-cooking/data?select=test.json.zip
train_url = "https://raw.githubusercontent.com/hchuhtala/Recipe-ML-App/f22de810927f2ed0c079c1b021cd757a056ef24e/data/train.json"
test_url = "https://raw.githubusercontent.com/hchuhtala/Recipe-ML-App/f22de810927f2ed0c079c1b021cd757a056ef24e/data/test.json"

In [3]:
#Store data in Pandas Data Frames
train_df = pd.read_json(train_url).set_index('id')
train_df.head(10)

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
6602,jamaican,"[plain flour, sugar, butter, eggs, fresh ginge..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli..."
3735,italian,"[sugar, pistachio nuts, white almond bark, flo..."
16903,mexican,"[olive oil, purple onion, fresh pineapple, por..."


In [4]:
test_df = pd.read_json(test_url).set_index('id')

Data Clean Up:

In [5]:
original_cuisine = train_df["cuisine"].unique()
original_cuisine_list = original_cuisine.tolist()
original_cuisine_list

['greek',
 'southern_us',
 'filipino',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'brazilian',
 'french',
 'japanese',
 'irish',
 'korean',
 'moroccan',
 'russian']

In [6]:
print("There are " + str(train_df.count()[0]) + " records in the raw data set")

There are 39774 records in the raw data set


In [7]:
# #dropping cuisines types that are not in both this data set and API used later in the app
cuisine_list = ['british', 'chinese', 'cajun_creole', 'french', 'greek', 'indian', 'irish', 'italian', 'japanese', 'korean', 'mexican', 'southern_us', 'spanish', 'thai', 'vietnamese']
train_clean_df = train_df[train_df.cuisine.isin(cuisine_list)]
train_clean_df.head()

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli..."


In [8]:
cuisine_list = train_clean_df["cuisine"].unique()
cuisine_list = cuisine_list.tolist()
cuisine_list

['greek',
 'southern_us',
 'indian',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'french',
 'japanese',
 'irish',
 'korean']

In [9]:
cuisine_df = pd.DataFrame(cuisine_list)
cuisine_df

,0
0,greek
1,southern_us
2,indian
3,spanish
4,italian
5,mexican
6,chinese
7,british
8,thai
9,vietnamese


In [10]:
print("We will be working with " + str(train_clean_df.count()[0]) + " records for our machine learning")

We will be working with 36716 records for our machine learning


In [11]:
type(train_clean_df.ingredients)

pandas.core.series.Series

In [12]:
train_clean_df['text'] = train_clean_df['ingredients'].apply(lambda x:','.join([str(i) for i in x]))
train_clean_df

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cuisine,ingredients,text
id,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground..."
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper..."
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho..."
...,...,...,...
29109,irish,"[light brown sugar, granulated sugar, butter, ...","light brown sugar,granulated sugar,butter,warm..."
11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b...","KRAFT Zesty Italian Dressing,purple onion,broc..."
2238,irish,"[eggs, citrus fruit, raisins, sourdough starte...","eggs,citrus fruit,raisins,sourdough starter,fl..."


In [13]:
codes, uniques = pd.factorize(train_clean_df.cuisine)
codes

array([ 0,  1,  2, ..., 13,  6,  5], dtype=int64)

In [14]:
train_clean_df['cat_id'] = codes
train_clean_df.head()

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cuisine,ingredients,text,cat_id
id,,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga...",0
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground...",1
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt",2
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper...",2
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho...",3


In [15]:
# import re
# pattern = re.compile(r'\s+')
# train_clean_df['text2'] = re.sub(pattern, '', train_clean_df['text'])

# # train_clean_df['text2'] = train_clean_df['text'].replace(" ", "")
# train_clean_df

In [16]:
docs=["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
]

docs1 = [["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
], ["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
]]

docs2 = pd.DataFrame({ 'text': docs})

docs3 = pd.DataFrame({ 'text': docs1})
docs3

,text
0,"[the house had a tiny little mouse, the cat sa..."
1,"[the house had a tiny little mouse, the cat sa..."


In [17]:
docs3['B'] = docs3['text'].apply(lambda x:','.join([str(i) for i in x]))
docs3

,text,B
0,"[the house had a tiny little mouse, the cat sa...","the house had a tiny little mouse,the cat saw ..."
1,"[the house had a tiny little mouse, the cat sa...","the house had a tiny little mouse,the cat saw ..."


In [18]:
train_clean_df.head()

,cuisine,ingredients,text,cat_id
id,,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga...",0
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground...",1
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt",2
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper...",2
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho...",3


Machine Learning:

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here 
# tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer=TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english') 

 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(train_clean_df.text)


In [20]:
# get the first vector out (for the first document) 
first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0] 
first_vector_tfidfvectorizer


<1x16835 sparse matrix of type '<class 'numpy.float64'>'
	with 29 stored elements in Compressed Sparse Row format>

In [21]:
# place tf-idf values in a pandas data frame 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df = df.sort_values(by=["tfidf"],ascending=False)
df.head(20)

,tfidf
beans feta,0.316228
onion seasoning,0.316228
lettuce black,0.308746
garlic pepper,0.241639
garbanzo beans,0.228044
garbanzo,0.227744
pepper purple,0.215162
grape tomatoes,0.203910
grape,0.196546
romaine lettuce,0.193073


In [22]:
features = tfidf_vectorizer.fit_transform(train_clean_df.text).toarray()
labels = train_clean_df.cat_id
features.shape

(36716, 16835)

Now, each of 4569 consumer complaint narratives is represented by 12633 features, representing the tf-idf score for different unigrams and bigrams.

In [23]:
train_clean_df['category_id'] = train_clean_df['cuisine'].factorize()[0]
category_id_df = train_clean_df[['cuisine', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'cuisine']].values)
train_clean_df.head()

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,cuisine,ingredients,text,cat_id,category_id
id,,,,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes...","romaine lettuce,black olives,grape tomatoes,ga...",0,0
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...","plain flour,ground pepper,salt,tomatoes,ground...",1,1
22213,indian,"[water, vegetable oil, wheat, salt]","water,vegetable oil,wheat,salt",2,2
13162,indian,"[black pepper, shallots, cornflour, cayenne pe...","black pepper,shallots,cornflour,cayenne pepper...",2,2
42779,spanish,"[olive oil, salt, medium shrimp, pepper, garli...","olive oil,salt,medium shrimp,pepper,garlic,cho...",3,3


In [24]:
category_to_id.items()

dict_items([('greek', 0), ('southern_us', 1), ('indian', 2), ('spanish', 3), ('italian', 4), ('mexican', 5), ('chinese', 6), ('british', 7), ('thai', 8), ('vietnamese', 9), ('cajun_creole', 10), ('french', 11), ('japanese', 12), ('irish', 13), ('korean', 14)])

In [33]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = features
y = labels
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_df = pd.DataFrame(X_new)
X_df.head()

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [49]:
for cuisine, category_id in sorted(category_to_id.items()):
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]

(array([0.02992311, 0.02992311, 0.03016477, ..., 0.05273441, 0.09550275,
        0.10672369]),
 array([0.86266481, 0.86266481, 0.8621169 , ..., 0.81837173, 0.75729476,
        0.74390545]))

In [39]:
for category_id in sorted(category_to_id.items()):
    print(category_id)

('british', 7)
('cajun_creole', 10)
('chinese', 6)
('french', 11)
('greek', 0)
('indian', 2)
('irish', 13)
('italian', 4)
('japanese', 12)
('korean', 14)
('mexican', 5)
('southern_us', 1)
('spanish', 3)
('thai', 8)
('vietnamese', 9)


In [51]:
from sklearn.feature_selection import chi2
import numpy as np
N = 10
for cuisine, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf_vectorizer.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(cuisine))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'british':
  . Most correlated unigrams:
. double
. puff
. treacle
. pastry
. haddock
. jam
. mincemeat
. suet
. marmite
. stilton
  . Most correlated bigrams:
. haddock fillets
. black treacle
. mixed spice
. raspberry jam
. malt vinegar
. milk eggs
. plain flour
. golden syrup
. stilton cheese
. beef drippings
# 'cajun_creole':
  . Most correlated unigrams:
. shrimp
. crawfish
. file
. okra
. sausage
. celery
. seasoning
. andouille
. creole
. cajun
  . Most correlated bigrams:
. creole mustard
. pepper cajun
. red beans
. file powder
. dried thyme
. green bell
. smoked sausage
. andouille sausage
. creole seasoning
. cajun seasoning
# 'chinese':
  . Most correlated unigrams:
. ginger
. spice
. sauce
. oyster
. hoisin
. shaoxing
. starch
. sesame
. chinese
. soy
  . Most correlated bigrams:
. rice wine
. light soy
. chinese spice
. spice powder
. shaoxing wine
. hoisin sauce
. oyster sauce
. corn starch
. sesame oil
. soy sauce
# 'french':
  . Most correlated unigrams:
. thyme
. di

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train_clean_df.text).toarray()
labels = df.category_id
features.shape

Model:

In [52]:
#https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(train_clean_df['text'], train_clean_df['cuisine'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\hboll\Anaconda3\envs\pythonadv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
cv_df.groupby('model_name').accuracy.mean()

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB()
#     , LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []

In [57]:
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [58]:
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [59]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 0.805126
MultinomialNB             0.686240
RandomForestClassifier    0.356602
Name: accuracy, dtype: float64

In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf_NB_TFIDF = MultinomialNB()
scores = model_selection.cross_val_score(clf_NB_TFIDF, train_tfidf, train["target"], cv=5, scoring="f1")
scores

In [ ]:
 import numpy as np
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 10))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X, y)
print(clf.predict(X[2:3]))
print(X)
print(X[2:3])
print(X[2])

In [ ]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X, y)
print(clf.predict(X[2:3]))

In [ ]:


# from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8)
processed_features = vectorizer.fit_transform(train_clean_df.ingredients).toarray()

In [ ]:
y = df["Outcome"]
target_names = ["negative", "positive"]

In [ ]:
X = df.drop("Outcome", axis=1)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 13 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(X_train_scaled, y_train)
print('k=13 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

In [ ]:
print(X.toarray())

In [ ]:
docs2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Build our text
text = """Yesterday I went fishing. I don't fish that often, 
so I didn't catch any fish. I was told I'd enjoy myself, 
but it didn't really seem that fun."""

vectorizer = CountVectorizer()

matrix = vectorizer.fit_transform(docs2.text)
matrix

In [ ]:
import pandas as pd

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
counts


Count ingredients 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(train_clean_df.text)


In [ ]:
import pandas as pd

counts = pd.DataFrame(matrix.toarray(),
                      columns=vectorizer.get_feature_names())
counts